<a href="https://colab.research.google.com/github/kenttw/kent-ai-learning-notebook/blob/master/Machine_Learning_Exercise/CreditCard/SimpleModel_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# if you have google drive permission then use this 

from google.colab import drive
drive.mount('/gdrive')
!ln -s   /gdrive/MyDrive/colab/Credit_Card/data/ /content/data

In [ ]:
!pip3 install gdown
!mkdir -p ./data/
import gdown


url = 'https://drive.google.com/uc?id=%s'%("1NZ8jMoqcCDpydHxWm6j415HwVS093l4U")
output = './data/X_sample.csv'
gdown.download(url, output, quiet=False)


url = 'https://drive.google.com/uc?id=%s'%("18w2LpyPbydD9qoVQ33GZI2LOjyRY2LaI")
output = './data/y_sample.csv'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=%s'%("1-0TD5rSfzG3frbgT4fseGBn-Dn5OpGKy")
output = './data/X_test.csv'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=%s'%("1-1p1dHQVV2JnGP4eMfzJsQwErum_wuMG")
output = './data/y_test.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NZ8jMoqcCDpydHxWm6j415HwVS093l4U
To: /content/data/X_sample.csv
48.9MB [00:00, 265MB/s]
Downloading...
From: https://drive.google.com/uc?id=18w2LpyPbydD9qoVQ33GZI2LOjyRY2LaI
To: /content/data/y_sample.csv
100%|██████████| 288k/288k [00:00<00:00, 41.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-0TD5rSfzG3frbgT4fseGBn-Dn5OpGKy
To: /content/data/X_test.csv
609MB [00:02, 225MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1p1dHQVV2JnGP4eMfzJsQwErum_wuMG
To: /content/data/y_test.csv
3.59MB [00:00, 206MB/s]


'./data/y_test.csv'

In [ ]:
import pandas as pd

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
df_X = pd.read_csv("./data/X_sample.csv")
df_X = df_X.set_index("txkey")

In [ ]:
df_y = pd.read_csv("./data/y_sample.csv")
df_y = df_y.set_index("txkey")

In [ ]:
import lightgbm as lgb


In [ ]:
from sklearn.model_selection import train_test_split
df_X_train, df_X_val = train_test_split(df_X, test_size=0.25, random_state=42)


In [ ]:
df_y_train = df_y.loc[df_X_train.index]
df_y_val = df_y.loc[df_X_val.index]

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_X_train, df_y_train)
lgb_eval = lgb.Dataset(df_X_val, df_y_val, reference=lgb_train)

In [ ]:
# specify your configurations as a dict
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
params['max_depth']=10

print('Starting training...')

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=20)



Starting training...
[1]	valid_0's binary_logloss: 0.670538
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's binary_logloss: 0.649209
[3]	valid_0's binary_logloss: 0.629096
[4]	valid_0's binary_logloss: 0.610069
[5]	valid_0's binary_logloss: 0.5921
[6]	valid_0's binary_logloss: 0.575188
[7]	valid_0's binary_logloss: 0.559077
[8]	valid_0's binary_logloss: 0.543858
[9]	valid_0's binary_logloss: 0.529395
[10]	valid_0's binary_logloss: 0.515652
[11]	valid_0's binary_logloss: 0.502559
[12]	valid_0's binary_logloss: 0.49015
[13]	valid_0's binary_logloss: 0.478375
[14]	valid_0's binary_logloss: 0.467075
[15]	valid_0's binary_logloss: 0.456233
[16]	valid_0's binary_logloss: 0.445863
[17]	valid_0's binary_logloss: 0.436013
[18]	valid_0's binary_logloss: 0.426567
[19]	valid_0's binary_logloss: 0.417556
[20]	valid_0's binary_logloss: 0.408935
Did not meet early stopping. Best iteration is:
[20]	valid_0's binary_logloss: 0.408935


# Validation with downsampleing data

In [ ]:
print('Starting predicting...')
# predict
y_pred = gbm.predict(df_X_val, num_iteration=gbm.best_iteration)
# eval
print('auc:', roc_auc_score(df_y_val, y_pred))

Starting predicting...
auc: 0.9655231831719928


In [ ]:
confusion_matrix(df_y_val, (y_pred > 0.5).astype(int))

array([[3543,  317],
       [ 213, 3569]])

In [ ]:
tn, fp, fn, tp  = confusion_matrix(df_y_val, (y_pred > 0.5).astype(int)).ravel() 

In [ ]:
tn, fp, fn, tp

(3543, 317, 213, 3569)

# Testing : verify with real distribution data

In [ ]:
X_test = pd.read_csv("./data/X_test.csv")

In [ ]:
X_test = X_test.set_index("txkey")

In [ ]:
y_test = pd.read_csv("./data/y_test.csv")

In [ ]:
y_test = y_test.set_index("txkey")

In [ ]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)


In [ ]:
y_test.shape

(380447, 1)

In [ ]:
print('auc:', roc_auc_score(y_test, y_pred))

auc: 0.9618565375213639


In [ ]:
cm = confusion_matrix(y_test, (y_pred > 0.5).astype(int), labels=[0,1])
cm

array([[341759,  33616],
       [   293,   4779]])

In [ ]:
th = .6
print("precision_score", precision_score(y_test,(y_pred > th).astype(int)))
print("recall_score",    recall_score(y_test,(y_pred > th).astype(int)))
print("f1_score",        f1_score(y_test,(y_pred > th).astype(int)))

precision_score 0.1468980422069667
recall_score 0.9112776025236593
f1_score 0.2530107291438581


In [ ]:
fnscore =  [ (fname,socre) for fname,socre in zip(gbm.feature_name(),gbm.feature_importance())]

In [ ]:
top_feature = sorted(fnscore, key=lambda tup: tup[1], reverse=True)[:25]